In [1]:
# Importing required libraries
import time
import pymongo
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient

In [2]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['Examen2']
collection = db['Candidatos']

In [3]:
# Base URL of the site to be analyzed
SITE_URL = 'https://alianzaverde.org.co/'

In [4]:
# Firefox web driver path
DRIVER_PATH = './geckodriver.exe'

In [5]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\ALFRED~1\AppData\Local\Temp/ipykernel_16596/1910320475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [6]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [7]:
# Getting HTML content for a particular news listing page
soup = make_request(browser, '/lideres-verdes/congreso-2022/senado-2022')

In [8]:
pages = soup.find(class_ = 'main-body')

In [9]:
sppbs = pages.find_all(class_ = 'sppb-wow', recursive = True)
print(len(sppbs))

9


In [10]:
# Finding and concatenating news cards
cards = sppbs[0].find_all(class_ = 'sppb-col-md-3') + sppbs[1].find_all(class_ = 'sppb-col-md-3') + sppbs[2].find_all(class_ = 'sppb-col-md-3') + sppbs[3].find_all(class_ = 'sppb-col-md-3')+ sppbs[4].find_all(class_ = 'sppb-col-md-3') + sppbs[5].find_all(class_ = 'sppb-col-md-3') + sppbs[6].find_all(class_ = 'sppb-col-md-3') + sppbs[7].find_all(class_ = 'sppb-col-md-3') + sppbs[8].find_all(class_ = 'sppb-col-md-3')
len(cards)

36

In [12]:
# Building a list with title and relative path of the news founded
data = []

for text in cards: 
        data.append({
            'Imagen': text.find('img', class_ = 'sppb-img-responsive').get('src'),
            'Descripcion': text.find('span', class_ = 'sppb-person-designation').get_text(),
            'Redes': text.find('ul', class_ = 'sppb-person-social').find('a')['href']
        })

In [13]:
data

[{'Imagen': '/images/Candidatos_2022/senado/2.jpg',
  'Descripcion': 'Abogada y filósofa, con un máster en Ciencias de la Administración Pública, en Alemania. Experta en gestión pública, descentralización, transparencia, buen gobierno, modernización del estado, cultura ciudadana y cooperación internacional, temas en los cuales se ha desempeñado alternativamente como servidora pública, consultora y docente.',
  'Redes': 'https://twitter.com/VivianaBarbere '},
 {'Imagen': '/images/Candidatos_2022/senado/3.jpg',
  'Descripcion': 'Nacida de seres excepcionales, en tiempos inciertos; es la mujer de retos resueltos. Desde el inicio de su carrera política, su liderazgo se enfoca en mantener la cultura de la equidad, la familia y el fortalecimiento económico, político y social de las regiones en Colombia',
  'Redes': 'https://www.facebook.com/sandraortizsenadora'},
 {'Imagen': '/images/Candidatos_2022/senado/5.jpg',
  'Descripcion': 'Destacado Senador de la República en el año 2010 fue discipl

In [14]:
# Storing extracted information for further analysis
collection.insert_many(data)